<a href="https://colab.research.google.com/github/banshee0716/Financial-Big-Data-Analysis/blob/master/%E9%87%91%E8%9E%8D%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90W12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 官方範例

In [30]:
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)

Artificial intelligence (AI) is a broad field encompassing various techniques that enable computers to mimic human intelligence.  There's no single "how it works" answer, as different AI approaches use different methods. However, we can break it down into key concepts:

**1. Data is King:**  AI systems learn from data.  The more data, and the higher the quality of that data, the better the AI system typically performs. This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms are the Recipes:** Algorithms are sets of instructions that tell the computer how to process and learn from the data. These algorithms are the core of AI, and different algorithms are suited for different tasks.

**3. Machine Learning (ML): A Major Approach:**  Machine learning is a subset of AI where the algorithm learns patterns from data without being explicitly programmed. Instead of relying on hard-coded rules, it identifies trends and relationships within 

## 金融分析程式碼

In [56]:
import yfinance as yf
import pandas as pd

# 定義股票代號
ticker = "2330.TW"

# 獲取股票資訊
stock = yf.Ticker(ticker)
hist = stock.history(period="6mo")

# 獲取最近一年的財報數據
financials = stock.financials
balance_sheet = stock.balance_sheet
cashflow = stock.cashflow

# 獲取股息收益率
dividends = stock.dividends
if not dividends.empty:
    dividend_yield = dividends.iloc[-1] / stock.history(period="1d")['Close'].iloc[-1] * 100
else:
    dividend_yield = 0.0

# 獲取每股盈餘 (EPS)
eps = stock.info.get('trailingEps', None)

# 獲取市盈率 (P/E Ratio)
pe_ratio = stock.info.get('trailingPE', None)

# 計算技術指標
# 簡單移動平均線 (SMA)
hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
hist['SMA_50'] = hist['Close'].rolling(window=50).mean()

# 相對強弱指數 (RSI)
delta = hist['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
hist['RSI'] = 100 - (100 / (1 + rs))

# 移動平均收斂發散指標 (MACD)
hist['EMA_12'] = hist['Close'].ewm(span=12, adjust=False).mean()
hist['EMA_26'] = hist['Close'].ewm(span=26, adjust=False).mean()
hist['MACD'] = hist['EMA_12'] - hist['EMA_26']
hist['Signal_Line'] = hist['MACD'].ewm(span=9, adjust=False).mean()

# 構建財務數據字典
financial_data = {
    "revenue": financials.loc['Total Revenue'].iloc[-1] / 1e9 if 'Total Revenue' in financials.index else None,  # 轉換為億
    "net_income": financials.loc['Net Income'].iloc[-1] / 1e9 if 'Net Income' in financials.index else None,  # 轉換為億
    "pe_ratio": pe_ratio,
    "eps": eps,
    "stock_price": hist['Close'].iloc[-1],
    "dividend_yield": dividend_yield,
    "SMA_20": hist['SMA_20'].iloc[-1],
    "SMA_50": hist['SMA_50'].iloc[-1],
    "RSI": hist['RSI'].iloc[-1],
    "MACD": hist['MACD'].iloc[-1],
    "Signal_Line": hist['Signal_Line'].iloc[-1],
}

print(financial_data)


{'revenue': nan, 'net_income': nan, 'pe_ratio': 24.592592, 'eps': 40.5, 'stock_price': 996.0, 'dividend_yield': 0.4016064257028112, 'SMA_20': 1036.3, 'SMA_50': 1025.74, 'RSI': 26.455026455026456, 'MACD': -7.626143602493812, 'Signal_Line': -1.097658088525212}


In [57]:
def generate_stock_analysis(ticker, financial_data):
    """
    使用 Gemini API 生成股票分析報告
    :param ticker: 股票代碼
    :param financial_data: 股票的關鍵財務數據（字典格式）
    :return: 生成的分析報告
    """
    # 定義 prompt 模板
    prompt = f"""我需要一份股票分析報告，針對以下公司數據進行全面分析，並提供專業的投資建議。以下是 {ticker} 的關鍵數據：

### 財務數據：
- 收入: {financial_data.get('revenue', 'N/A')} 億美元
- 淨利潤: {financial_data.get('net_income', 'N/A')} 億美元
- 每股盈餘 (EPS): {financial_data.get('eps', 'N/A')}
- 市盈率 (P/E Ratio): {financial_data.get('pe_ratio', 'N/A')}
- 股價: {financial_data.get('stock_price', 'N/A')} 美元
- 股息收益率: {financial_data.get('dividend_yield', 'N/A')}%

### 技術指標：
- 簡單移動平均線 (SMA-20): {financial_data.get('SMA_20', 'N/A')} 美元
- 簡單移動平均線 (SMA-50): {financial_data.get('SMA_50', 'N/A')} 美元
- 相對強弱指數 (RSI): {financial_data.get('RSI', 'N/A')}
- 移動平均收斂發散指標 (MACD): {financial_data.get('MACD', 'N/A')}
- 信號線 (Signal Line): {financial_data.get('Signal_Line', 'N/A')}

### 任務要求：
你是一位專業的投資經理人，請根據上述財務與技術數據撰寫一份專業的分析報告，報告需包含以下內容：
1. **公司財務健康狀況的詳細評論**，包括收入、淨利潤、市盈率等關鍵指標的解讀。
2. **技術指標的分析**，解釋移動平均線、RSI、MACD 等指標的趨勢和含義。
3. **未來增長潛力的判斷**，基於財務數據和技術分析提供專業見解。
4. **投資建議**，具體說明是否適合短期或長期投資，並解釋理由。
5. 如果適用，對潛在風險進行評估，並提出應對策略。

    """

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")  # 使用 Gemini 模型
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"生成失敗: {e}"

sample_data = financial_data

analysis_report = generate_stock_analysis(ticker, sample_data)


In [58]:
def display_markdown_output(ticker, analysis_report):
    """
    直接在輸出中以 Markdown 語法顯示股票分析報告
    """
    markdown_output = f"""
# {ticker} 股票分析報告

## 摘要
以下是基於公司財務數據的專業分析報告。

## 詳細報告
{analysis_report}
    """
    # 打印 Markdown 格式內容
    print(markdown_output)

# 測試輸出
display_markdown_output(ticker, analysis_report)



# 2330.TW 股票分析報告

## 摘要
以下是基於公司財務數據的專業分析報告。

## 詳細報告
## 2330.TW 股票分析報告

**日期:** 2024年10月27日

**分析師:** AI 投資顧問

**引言:**

本報告針對台灣上市公司 2330.TW (請提供公司名稱) 進行全面分析，基於提供的財務和技術數據，評估其投資價值及潛在風險。由於提供的收入和淨利潤數據缺失 (顯示為 nan)，本報告將主要依據提供的每股盈餘 (EPS)、市盈率 (P/E Ratio)、股價、股息收益率以及技術指標進行分析，並指出數據缺失對分析的限制。


**1. 公司財務健康狀況評論:**

提供的數據顯示 2330.TW 的 EPS 為 40.5，市盈率為 24.59。這意味著投資者願意支付其每股盈餘的 24.59 倍來購買該公司的股票。相較於市場平均市盈率 (需要參考市場數據，本報告無法提供)，24.59 的市盈率需要進一步與同業比較才能判斷其高低。缺乏收入和淨利潤數據，難以評估公司的盈利能力和整體財務健康狀況。  股息收益率為 0.40%，相對較低，暗示股價增長可能是投資者主要期待的回報來源。  **數據缺失嚴重限制了對公司財務健康狀況的全面評估。**  我們需要完整的財務報表才能得出更準確的結論。


**2. 技術指標分析:**

* **移動平均線 (SMA):** 20 日 SMA (1036.3 美元) 低於 50 日 SMA (1025.74 美元)，呈現輕微的看跌訊號。  這表示短期平均價格低於長期平均價格，價格可能存在下跌趨勢。然而，差距很小，趨勢並不明顯。

* **相對強弱指數 (RSI):** RSI 為 26.46，低於 30 的超賣區間。這通常被認為是潛在買入訊號，表示股票可能被低估，存在反彈的可能性。

* **移動平均收斂發散指標 (MACD):** MACD 為 -7.63，低於信號線 (-1.10)，呈現看跌訊號。  這表明價格動能減弱，並且存在持續下跌的風險。

**綜合技術指標分析：** 技術指標呈現相互矛盾的訊號。RSI 指示超賣，暗示潛在的反彈，而 SMA 和 MACD 則顯示看跌趨勢。  需要更多數據和更深入的分析才能確認價格走勢。


**3. 未來增長潛力判斷:**

由於缺乏關鍵財務數據 (收入和淨利潤